# GPU 메모리 초기화

In [ ]:
!/opt/bin/nvidia-smi

Wed Mar 23 12:51:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!ps -aux|grep python

root          35  0.3  0.0      0     0 ?        Z    13:49   0:03 [python3] <defunct>
root         154 10.1  0.2 203832 60212 ?        Sl   14:08   0:00 /usr/bin/python3 /usr/local/bin/jupyter-notebook --ip="172.28.0.2" --port=9000 --FileContentsManager.root_dir="/" --MappingKernelManager.root_dir="/content"
root         160 20.2  0.3 533696 92784 ?        Ssl  14:08   0:01 /usr/bin/python3 -m ipykernel_launcher -f /root/.local/share/jupyter/runtime/kernel-17e21149-8b98-4092-a7bc-44b5d13d387a.json
root         182  0.0  0.0  93584 14388 ?        Sl   14:08   0:00 /usr/bin/python3 /usr/local/lib/python3.7/dist-packages/debugpy/adapter --for-server 43387 --host 127.0.0.1 --port 17899 --server-access-token 5c29ea974cf9569764fcc46e7c7e024b5a9890a18f12df246ab0560f826ca5df
root         199  0.0  0.0  39200  6508 ?        S    14:08   0:00 /bin/bash -c ps -aux|grep python
root         201  0.0  0.0  38576  4956 ?        S    14:08   0:00 grep python


In [ ]:
!kill -35 36 61 85 107 144 146

# Importing packages

In [ ]:
!pip install -q transformers  rouge-score

     |████████████████████████████████| 3.8 MB 32.7 MB/s 
     |████████████████████████████████| 596 kB 62.1 MB/s 
     |████████████████████████████████| 6.5 MB 59.9 MB/s 
     |████████████████████████████████| 67 kB 6.2 MB/s 
     |████████████████████████████████| 895 kB 76.2 MB/s 


In [ ]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd 
import transformers
from torch.utils.data import Dataset, DataLoader 
from transformers import AutoTokenizer, AutoModel
from transformers.optimization import get_linear_schedule_with_warmup
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"# available GPUs : {torch.cuda.device_count()}")
    print(f"GPU name : {torch.cuda.get_device_name()}")
else:
    device = torch.device("cpu")
print(device)

# available GPUs : 1
GPU name : Tesla T4
cuda


# 데이터 사용을 위한 google drive import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd "/content/drive/MyDrive/기업과제4"

[Errno 2] No such file or directory: '/content/drive/MyDrive/기업과제4'
/content


# Traing data load

In [ ]:
total_df_ver1 = pd.read_json('/content/drive/MyDrive/Classroom/AI심화과정/data/train_bdf_ko_spt_total.json')

In [ ]:
total_df_ver1.head()

,sents,docs,y
0,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다.,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,0
1,"신승재 사장은유상철 감독을 비롯한 코칭스텝, 선수단 구성이 마무리 된 만큼 구성원 ...",전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,1
2,유상철 감독은구봉산의 정기를 받아 년을 전남드래곤즈의 해로 만들겠다며 각오를 다졌다.,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,0
3,한편 전남은 선수들도 추가 영입했다.,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,1
4,공격력 강화를 위해 페체신 대체자로 중국 리그에서 활약한 하태균도 영입했다.,전남드래곤즈사장 신승재는 지난 일 구봉산 해맞이 행사를 통해 새해 각오를 다졌다. ...,0


In [ ]:
total_df_ver2 = pd.read_json('/content/drive/MyDrive/Classroom/AI심화과정/data/clova_summary_data_5077.json')

In [ ]:
total_df_ver2.head()

,sents,docs,y
0,일본 국가대표 미드필더 카마다 다이치(25)가 아인트라흐트 프랑크푸르트에 잔류한 이...,일본 국가대표 미드필더 카마다 다이치(25)가 아인트라흐트 프랑크푸르트에 잔류한 이...,0
1,올여름 이적 시장에서 카마다를 눈독 들이는 팀이 많았다.,일본 국가대표 미드필더 카마다 다이치(25)가 아인트라흐트 프랑크푸르트에 잔류한 이...,0
2,"독일 빌트가 지난달 27일 ""프랑크푸르트 공격형 미드필더 카마다의 프리미어리그 이적...",일본 국가대표 미드필더 카마다 다이치(25)가 아인트라흐트 프랑크푸르트에 잔류한 이...,1
3,그의 대리인인 로베르토 쓰카다가 몇 주 전부터 영국에 체류하고 있다.,일본 국가대표 미드필더 카마다 다이치(25)가 아인트라흐트 프랑크푸르트에 잔류한 이...,0
4,프리미어리그 다수 팀이 관심을 드러냈다.,일본 국가대표 미드필더 카마다 다이치(25)가 아인트라흐트 프랑크푸르트에 잔류한 이...,0


# 학습 관련 변수 정의

In [ ]:
#모델 설정 관련 파라미터
model_name = "klue/bert-base"
#model_name = "klue/roberta-small"
#model_name = "klue/roberta-base"
MAX_LEN = 512
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 8
LEARNING_RATE = 1e-05
neg_multiplier = 2
print_n_steps = 1000
EPOCHS = 5

# DataLoader 정의

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/289 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/425 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/243k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
# Create a Data Loader Class
class Sentence_Docs(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len
        
    def __getitem__(self, index):
        sentence = str(self.data.iloc[index].sents)
        sentence = " ".join(sentence.split())

        document = str(self.data.iloc[index].docs)
        document = " ".join(document.split())

        inputs = self.tokenizer.batch_encode_plus(
            [sentence, document], 
            add_special_tokens=True,
            max_length=self.max_len,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']

        #input의 값 중 'input_ids', 'attention_mask'을 사용하여 문장과 문서를 임베딩, 레이블 값을 사용
        #아웃풋은 dict 형식
        return {
            'sent_ids': torch.tensor(ids[0], dtype=torch.long),
            'doc_ids': torch.tensor(ids[1], dtype=torch.long),
            'sent_mask': torch.tensor(mask[0], dtype=torch.long),
            'doc_mask': torch.tensor(mask[1], dtype=torch.long),
            'targets': torch.tensor([self.data.iloc[index].y], dtype=torch.long)
        } 
    
    def __len__(self):
        return self.len

# Train 함수 정의

In [ ]:
# get mean pooling for sentence bert models 
# ref https://www.sbert.net/examples/applications/computing-embeddings/README.html#sentence-embeddings-with-transformers
def mean_pooling(model_output, attention_mask):
    #print(model_output[0] == model_output["last_hidden_state"])
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
# Creating the customized model, by adding a drop out and a dense layer on top of distil bert to get the final output for the model. 
# Note that different sentence transformer models may have different in_feature sizes
class SentenceBertClass(torch.nn.Module):
    def __init__(self, model_name=model_name, in_features=768):
        super(SentenceBertClass, self).__init__()
        self.l1 = AutoModel.from_pretrained(model_name)
        self.pre_classifier = torch.nn.Linear(in_features*3, 768)
        self.dropout = torch.nn.Dropout(0.3)
        self.classifier = torch.nn.Linear(768, 1)
        self.classifierSigmoid = torch.nn.Sigmoid()

    def forward(self, sent_ids, doc_ids, sent_mask, doc_mask):

        sent_output = self.l1(input_ids=sent_ids, attention_mask=sent_mask)
        sentence_embeddings = mean_pooling(sent_output, sent_mask)

        doc_output = self.l1(input_ids=doc_ids, attention_mask=doc_mask)
        doc_embeddings = mean_pooling(doc_output, doc_mask)

        # elementwise product of sentence embs and doc embs
        combined_features = sentence_embeddings * doc_embeddings

        # Concatenate input features and their elementwise product
        concat_features = torch.cat((sentence_embeddings, doc_embeddings, combined_features), dim=1)   
        
        pooler = self.pre_classifier(concat_features) 
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        output = self.classifierSigmoid(output) 

        return output

In [ ]:
#acc_step_holder, loss_step_holder = [], []

def train(model, epoch):
    global scheduler, loss_function
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    model.to(device)
    for _,data in tqdm(enumerate(training_loader, 0)):
        sent_ids = data['sent_ids'].to(device, dtype = torch.long)
        doc_ids = data['doc_ids'].to(device, dtype = torch.long)
        sent_mask = data['sent_mask'].to(device, dtype = torch.long)
        doc_mask = data['doc_mask'].to(device, dtype = torch.long) 
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(sent_ids, doc_ids, sent_mask, doc_mask)
        loss = loss_function(outputs, targets)
        tr_loss += loss.item()
        n_correct += torch.count_nonzero(targets == (outputs > 0.5)).item()

        nb_tr_steps += 1
        nb_tr_examples+=targets.size(0)
        
        if _%print_n_steps==0:
            loss_step = tr_loss/nb_tr_steps
            accu_step = (n_correct*100)/nb_tr_examples 
            print(str(_* train_params["batch_size"]) + "/" + str(len(train_data)) + " - Steps. Acc ->", accu_step, "Loss ->", loss_step)
            #acc_step_holder.append(accu_step), loss_step_holder.append(loss_step)
        optimizer.zero_grad()
        loss.backward()
        # # When using GPU
        optimizer.step()
        scheduler.step()

    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct*100)/nb_tr_examples}')
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return

# model, optimizer, scheduler 초기화 함수 정의

In [ ]:
def initializer(train_dataloader, epochs=2):
    
    model = SentenceBertClass(model_name=model_name)

    optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
    
    total_steps = len(train_dataloader) * epochs
    print(f"Total train steps with {epochs} epochs: {total_steps}")

    scheduler = get_linear_schedule_with_warmup(
                                                optimizer, 
                                                num_warmup_steps=0,
                                                num_training_steps=total_steps
                                                )

    return model, optimizer, scheduler

# Validate 함수 정의

In [ ]:
def validate_model(model, testing_loader):
    global loss_function

    model.eval()

    n_correct = 0; n_wrong = 0; total = 0;  tr_loss = 0; nb_tr_steps = 0 ; nb_tr_examples = 0;
    with torch.no_grad():
        for _, data in enumerate(testing_loader, 0): 
            
            sent_ids = data['sent_ids'].to(device, dtype = torch.long)
            doc_ids = data['doc_ids'].to(device, dtype = torch.long)
            sent_mask = data['sent_mask'].to(device, dtype = torch.long)
            doc_mask = data['doc_mask'].to(device, dtype = torch.long) 
            targets = data['targets'].to(device, dtype = torch.float)  

            outputs = model(sent_ids, doc_ids, sent_mask, doc_mask) 
            loss = loss_function(outputs, targets)
            tr_loss += loss.item()
            
            n_correct += torch.count_nonzero(targets == (outputs > 0.5)).item()

            nb_tr_steps += 1
            nb_tr_examples+=targets.size(0)
            
            if _%print_n_steps==0:
                loss_step = tr_loss/nb_tr_steps
                accu_step = (n_correct*100)/nb_tr_examples 
                print(str(_* test_params["batch_size"]) + "/" + str(len(train_df)) + " - Steps. Acc ->", accu_step, "Loss ->", loss_step)

             
    epoch_loss = tr_loss/nb_tr_steps
    epoch_accu = (n_correct*100)/nb_tr_examples
    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")
    
    return epoch_accu

# 모델 학습

In [ ]:
ai_hub_p = total_df_ver1.loc[total_df_ver1.y == 1]
ai_hub_n = total_df_ver1.loc[total_df_ver1.y == 0].sample(len(ai_hub_p)*neg_multiplier, random_state=42)

clova_p = total_df_ver2.loc[total_df_ver2.y == 1]
clova_n = total_df_ver2.loc[total_df_ver2.y == 0].sample(len(clova_p)*neg_multiplier, random_state=42)

In [ ]:
train_data = pd.concat([ai_hub_p, ai_hub_n, clova_p, clova_n])

training_set = Sentence_Docs(train_data, tokenizer, MAX_LEN)
    
train_params = {'batch_size': TRAIN_BATCH_SIZE, 'shuffle': True}#, 'num_workers': 0}
    
training_loader = DataLoader(training_set, **train_params)

model, optimizer, scheduler = initializer(training_loader, EPOCHS)
loss_function = torch.nn.BCELoss()  # https://discuss.pytorch.org/t/loss-function-for-binary-classification-with-pytorch/26460

for epoch in range(EPOCHS):
  train(model, epoch)
  if epoch >= 2:
    torch.save(model.state_dict(), f"Matchsum_model_{epoch}_final_0323")

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Total train steps with 5 epochs: 51765


0it [00:00, ?it/s]

0/82818 - Steps. Acc -> 87.5 Loss -> 0.6443257927894592
8000/82818 - Steps. Acc -> 74.61288711288711 Loss -> 0.5188938583974
16000/82818 - Steps. Acc -> 75.76211894052973 Loss -> 0.4996729460933517
24000/82818 - Steps. Acc -> 76.56197934021992 Loss -> 0.4855370550990224
32000/82818 - Steps. Acc -> 77.01824543864034 Loss -> 0.47834746387825106
40000/82818 - Steps. Acc -> 77.40201959608078 Loss -> 0.47262575791564326
48000/82818 - Steps. Acc -> 77.74745875687385 Loss -> 0.4663683848629056
56000/82818 - Steps. Acc -> 77.86566204827882 Loss -> 0.4630384688449066
64000/82818 - Steps. Acc -> 78.1917885264342 Loss -> 0.45870816716830026
72000/82818 - Steps. Acc -> 78.38573491834241 Loss -> 0.45605922519909037
80000/82818 - Steps. Acc -> 78.56089391060894 Loss -> 0.4532784902050297
The Total Accuracy for Epoch 0: 78.6242121277983
Training Loss Epoch: 0.45259232216585804
Training Accuracy Epoch: 78.6242121277983


...

0/82818 - Steps. Acc -> 100.0 Loss -> 0.18068182468414307
8000/82818 - Steps. Acc -> 87.92457542457542 Loss -> 0.2849565509531822
16000/82818 - Steps. Acc -> 88.07471264367815 Loss -> 0.2829591103937423
24000/82818 - Steps. Acc -> 88.09563478840387 Loss -> 0.2811324365428652
32000/82818 - Steps. Acc -> 87.96550862284428 Loss -> 0.28402656503545454
40000/82818 - Steps. Acc -> 87.999900019996 Loss -> 0.28172819126872994
48000/82818 - Steps. Acc -> 87.9728378603566 Loss -> 0.2812485015348011
56000/82818 - Steps. Acc -> 88.02135409227253 Loss -> 0.27994124071527293
64000/82818 - Steps. Acc -> 88.00931133608299 Loss -> 0.28047794639176565
72000/82818 - Steps. Acc -> 87.96939228974558 Loss -> 0.28141946934858675
80000/82818 - Steps. Acc -> 87.96745325467454 Loss -> 0.28168824875834203
The Total Accuracy for Epoch 2: 87.98691105798257
Training Loss Epoch: 0.28154050999499247
Training Accuracy Epoch: 87.98691105798257


0it [00:00, ?it/s]

0/82818 - Steps. Acc -> 87.5 Loss -> 0.24662962555885315
8000/82818 - Steps. Acc -> 92.45754245754246 Loss -> 0.1967535994758087
16000/82818 - Steps. Acc -> 92.460019990005 Loss -> 0.19196289500662836
24000/82818 - Steps. Acc -> 92.41919360213262 Loss -> 0.19038954311157852
32000/82818 - Steps. Acc -> 92.34566358410397 Loss -> 0.1911032978678413
40000/82818 - Steps. Acc -> 92.30653869226155 Loss -> 0.19080147420364743
48000/82818 - Steps. Acc -> 92.29920013331112 Loss -> 0.19166003887413868
56000/82818 - Steps. Acc -> 92.27074703613769 Loss -> 0.19250921302993443
64000/82818 - Steps. Acc -> 92.34314460692413 Loss -> 0.19120213506792666
72000/82818 - Steps. Acc -> 92.34112876347072 Loss -> 0.19078650410890718
80000/82818 - Steps. Acc -> 92.3057694230577 Loss -> 0.19121822350794995
The Total Accuracy for Epoch 3: 92.31568016614746
Training Loss Epoch: 0.19086037897082295
Training Accuracy Epoch: 92.31568016614746


0it [00:00, ?it/s]

0/82818 - Steps. Acc -> 100.0 Loss -> 0.08546830713748932
8000/82818 - Steps. Acc -> 94.89260739260739 Loss -> 0.13458114890295633
16000/82818 - Steps. Acc -> 94.8088455772114 Loss -> 0.1349954524371917
24000/82818 - Steps. Acc -> 94.93918693768744 Loss -> 0.13324014509316462
32000/82818 - Steps. Acc -> 94.94501374656336 Loss -> 0.13241449100367741
40000/82818 - Steps. Acc -> 95.0134973005399 Loss -> 0.13084124424806734
48000/82818 - Steps. Acc -> 94.99666722212964 Loss -> 0.13107040107866802
56000/82818 - Steps. Acc -> 95.0560634195115 Loss -> 0.13039389312830807
64000/82818 - Steps. Acc -> 95.12404699412573 Loss -> 0.12914810032303714
72000/82818 - Steps. Acc -> 95.14498389067882 Loss -> 0.1286399728897566
80000/82818 - Steps. Acc -> 95.16923307669234 Loss -> 0.12797413989997342
The Total Accuracy for Epoch 4: 95.18582916757227
Training Loss Epoch: 0.12747488451720768
Training Accuracy Epoch: 95.18582916757227


# Inference: PREDICT SUMMARY 추출

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import os 
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from transformers import AutoTokenizer, AutoModel 

In [ ]:
#model path .. modify to location of trained model file 
model_path = "/content/drive/MyDrive/기업과제4/Matchsum_model_4_final_0323"

In [ ]:
test_df = pd.read_csv('/content/drive/MyDrive/기업과제4/test_df_p_4.csv')

In [ ]:
test_df["PREDICT SUMMARY"] = ""
test_df["SCORE"] = ""

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'

extractive_model = SentenceBertClass() 
extractive_model.load_state_dict(torch.load(model_path, map_location=torch.device(device) ))
extractive_model.eval();

Downloading:   0%|          | 0.00/424M [00:00<?, ?B/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# tokenize text as required by BERT based models
def get_tokens(text, tokenizer):
  inputs = tokenizer.batch_encode_plus(
            text, 
            add_special_tokens=True,
            max_length=512,
            padding="max_length",
            return_token_type_ids=True,
            truncation=True
        )
  ids = inputs['input_ids']
  mask = inputs['attention_mask']
  return ids, mask

In [ ]:
# get predictions given some an array of sentences and their corresponding documents
def predict(model,sents, doc):
  sent_id, sent_mask = get_tokens(sents,tokenizer)
  sent_id, sent_mask = torch.tensor(sent_id, dtype=torch.long),torch.tensor(sent_mask, dtype=torch.long)
 
  doc_id, doc_mask = get_tokens([doc],tokenizer)
  doc_id, doc_mask = doc_id * len(sents), doc_mask* len(sents)
  doc_id, doc_mask = torch.tensor(doc_id, dtype=torch.long),torch.tensor(doc_mask, dtype=torch.long)

  preds = model(sent_id, doc_id, sent_mask, doc_mask)
  return preds

In [ ]:
def summarize(doc, model, min_sentence_length=14, top_k=3, batch_size=3):
  doc_sentences = []
  for sent in doc.split("\t"):
    if len(sent) > min_sentence_length: 
      doc_sentences.append(str(sent))
  
  doc_id, doc_mask = get_tokens([doc],tokenizer)
  doc_id, doc_mask = doc_id * batch_size, doc_mask* batch_size
  doc_id, doc_mask = torch.tensor(doc_id, dtype=torch.long),torch.tensor(doc_mask, dtype=torch.long)

  scores = [] 
  # run predictions using some batch size
  for i in range(int(len(doc_sentences) / batch_size) + 1):
    batch_start = i*batch_size  
    batch_end = (i+1) * batch_size if (i+1) * batch_size < len(doc) else len(doc)-1
    batch = doc_sentences[batch_start: batch_end]
    if batch:
      preds = predict(model, batch, doc)
      scores = scores + preds.tolist()
 
  sent_pred_list = [{"sentence": doc_sentences[i], "score": scores[i][0], "index":i} for i in range(len(doc_sentences))]
  sorted_sentences = sorted(sent_pred_list, key=lambda k: k['score'], reverse=True)

  if len(sorted_sentences) < top_k:
    sorted_result = sorted_sentences
    sorted_result = sorted(sorted_result, key=lambda k: k['index'])
  else:
    sorted_result = sorted_sentences[:top_k]
    sorted_result = sorted(sorted_result, key=lambda k: k['index']) 
  
  summary = [ x["sentence"] for x in sorted_result]
  summary = "\t".join(summary)

  return summary

In [ ]:
test_df.head()

,TITLE,CONTENT,LABEL,LABEL_CNT,PREDICT SUMMARY,SCORE
0,"'멀티골' 살라, 에버턴전 4-1 완승에도 분노했다...왜?",모하메드 살라가 잉글리시 프리미어리그(EPL) 무대에서 최고의 자리를 유지하고 있는...,"영국 매체 '리버풀 에코'는 2일(한국시간) ""위르겐 클롭 감독은 에버턴과의 경기에...",3,,
1,[리그1 현장 리포트] ‘7발롱’ 메시와 PSG의 명성에 어울리지 않은 경기력,저조한 경기력 때문이었을까?\t파리 생제르맹(PSG) 리오넬 메시의 표정은 어두웠다...,이번 경기는 메시가 7번째 발롱도르 수상 후 치러진 경기였기에 이목이 쏠렸다.\t'...,3,,
2,"토트넘 로메로, 내년 2월까지 아웃...'산체스-다이어 봐야 해?'",크리스티안 로메로를 2월까지 그라운드에서 볼 수 없다.\t토트넘 홋스퍼엔 엄청난 악...,"영국 매체 '디 애슬레틱'은 2일(한국시간) ""로메로는 부상으로 장기간 경기에 출전...",3,,
3,"이니에스타, “일본 축구 즐겁다, 빠르게 성장 중” 찬사",빗셀 고베 안드레스 이니에스타(37)가 일본 생활에 만족감을 드러냈다.\t스페인과 ...,스페인과 바르셀로나 전설인 이니에스타는 2018년 7월 FC바르셀로나와 결별 후 일...,3,,
4,"'듣긴 할까?' 맨유, 팬 자문위원회 출범...""서포터 의견 항상 경청할 것""",맨체스터 유나이티드가 팬들과의 관계를 개선하기 위해 팬 자문위원회를 출범시켰다.\t...,"영국 매체 '더 선'은 1일(현지시간) ""맨유는 지난 시즌 올드 트래포드에서 큰 시...",3,,


In [ ]:
from tqdm import tqdm
tqdm.pandas()

In [ ]:
test_df["PREDICT SUMMARY"] = test_df["CONTENT"].progress_apply(lambda x: summarize(x, extractive_model, min_sentence_length=14, top_k=3, batch_size=16))

In [ ]:
test_df.to_csv('기업과제4_9팀_sports_news_data.csv')